<a href="https://colab.research.google.com/github/narendra974/AIMLOPS_IISC/blob/main/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practical AI and MLOps : Assignment 3


### Download the dataset.

Download the dataset attached with the assignment and store it in a pandas dataframe `df`. You are free to change the names as you like. You can split the datasets using `train_test_split` function from the `scikit-learn` library.

**1st dataset:** (df) For problem 1

In [26]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize

from keras.models import Sequential
from keras.layers import Dense,Activation, Input
from keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from tensorflow.keras import layers

In [3]:

df = pd.read_csv('Iris.csv')
df.head(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


## Problem 1 (5 marks)


You have to design and implement neural network models for multi-class classification using both a Multi-Layer Perceptron (MLP) and a Convolutional Neural Network (CNN). The dataset you will be working with contains samples from multiple classes. You need to experiment with different activation functions and loss functions for both models to find the best combinations for this task.

Instructions:

1. Load the multi-class dataset.

2. Preprocess the dataset as needed, including data splitting and normalization.

3. **Implement an MLP model**

Implement an MLP model with the following specifications:

*   Input layer with an appropriate number of neurons based on the dataset's features.
*   At least one hidden layer with a flexible number of neurons (you can experiment with this).
*   An output layer with neurons corresponding to the number of classes in the dataset.

Train the MLP model using the following settings:
*   Use two different activation functions for the hidden layers (e.g., ReLU and Sigmoid).
*   Use at least two different loss functions (e.g., Cross-Entropy and Mean Squared Error).
*   Experiment with various hyperparameters like learning rate, batch size, and the number of hidden neurons.

For each combination of activation function and loss function, train the model and evaluate its performance on the validation set using appropriate metrics (e.g., accuracy, F1-score).

Report the following for each combination:
*   Accuracy on the validation set.
*   F1-score on the validation set.
*   Confusion Matrix.

4. **Implement a CNN model**

Implement a 2D CNN model with the following specifications:

*   Convolutional layers with appropriate filters and kernel sizes.
*   At least one fully connected (dense) layer.
*   An output layer with neurons corresponding to the number of classes in the dataset.

Train the CNN model using the following settings:
*   Use two different activation functions for the convolutional and dense layers (e.g., ReLU and Tanh).
*   Use at least two different loss functions (e.g., Cross-Entropy and Categorical Hinge Loss).
*   Experiment with various hyperparameters like learning rate, batch size, and the number of filters.

For each combination of activation function and loss function, train the model and evaluate its performance on the validation set using appropriate metrics (e.g., accuracy, F1-score).

Report the following for each combination:
*   Accuracy on the validation set.
*   F1-score on the validation set.
*   Confusion Matrix.

In [4]:
print(df["Species"].unique())

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


In [5]:
df.loc[df["Species"]=="Iris-setosa","Species"]=0
df.loc[df["Species"]=="Iris-versicolor","Species"]=1
df.loc[df["Species"]=="Iris-virginica","Species"]=2

df=df.iloc[np.random.permutation(len(df))]
print(df.head())

      Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm Species
72    73            6.3           2.5            4.9           1.5       1
35    36            5.0           3.2            1.2           0.2       0
14    15            5.8           4.0            1.2           0.2       0
134  135            6.1           2.6            5.6           1.4       2
53    54            5.5           2.3            4.0           1.3       1


In [8]:
X=df.iloc[:,1:5].values
y=df.iloc[:,5].values

X_normalized=normalize(X,axis=0)

#Creating 80% train, 20% test data

total_length=len(df)
train_length=int(0.8*total_length)
test_length=int(0.2*total_length)

X_train=X_normalized[:train_length]
X_test=X_normalized[train_length:]
y_train=y[:train_length]
y_test=y[train_length:]

print("Length of train set x:",X_train.shape[0],"y:",y_train.shape[0])
print("Length of test set x:",X_test.shape[0],"y:",y_test.shape[0])

y_train=to_categorical(y_train,num_classes=3)
y_test= to_categorical(y_test,num_classes=3)
print("Shape of y_train",y_train.shape)
print("Shape of y_test",y_test.shape)

Length of train set x: 120 y: 120
Length of test set x: 30 y: 30
Shape of y_train (120, 3)
Shape of y_test (30, 3)


In [23]:
def create_model(no_hn, af, lf, lr, bs):

  model=Sequential()
  # model.add(Dense(1000,input_dim=4,activation='relu'))
  model.add(Input(shape=(4,)))
  model.add(Dense(no_hn, activation=af))
  model.add(Dense(3,activation='softmax'))
  opt = Adam(learning_rate=lr)
  model.compile(loss=lf, optimizer=opt,metrics=['accuracy'])
  model.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size=bs,epochs=20,verbose=0)

  prediction=model.predict(X_test)
  length=len(prediction)
  y_label=np.argmax(y_test,axis=1)
  predict_label=np.argmax(prediction,axis=1)

  acc = np.sum(y_label==predict_label)/length
  f1 = f1_score(y_label, predict_label, average='macro')
  conf_mat = confusion_matrix(y_label, predict_label)

  return acc, f1, conf_mat


In [24]:
acc, f1, conf_mat = create_model(10, 'relu', 'categorical_crossentropy', 0.01, 5)
print(acc)
print(f1)
print(conf_mat)

1/1 [==============================] - 0s 68ms/step
0.9
0.9011764705882354
[[ 9  0  0]
 [ 0  7  0]
 [ 0  3 11]]


In [25]:
# no_hn, af, lf, lr, bs
for no_hn in [5, 10, 15, 20]:
  for af in ['relu', 'sigmoid']:
    for lf in ['categorical_crossentropy', 'mean_squared_error']:
      for lr in [0.05, 0.1, 0.15]:
        for bs in [5, 10, 20]:
          acc, f1, conf = create_model(no_hn, af, lf, lr, bs)
          print('no_hn:{},  af:{}, lf:{}, lr:{}, bs:{}, acc:{}, f1:{}'.format(no_hn, af, lf, lr, bs, acc, f1))

1/1 [==============================] - 0s 68ms/step
no_hn:5,  af:relu, lf:categorical_crossentropy, lr:0.05, bs:5, acc:0.9333333333333333, f1:0.9326923076923076
1/1 [==============================] - 0s 62ms/step
no_hn:5,  af:relu, lf:categorical_crossentropy, lr:0.05, bs:10, acc:0.9666666666666667, f1:0.9628647214854111
1/1 [==============================] - 0s 85ms/step
no_hn:5,  af:relu, lf:categorical_crossentropy, lr:0.05, bs:20, acc:0.9666666666666667, f1:0.9654320987654321
1/1 [==============================] - 0s 53ms/step
no_hn:5,  af:relu, lf:categorical_crossentropy, lr:0.1, bs:5, acc:0.9666666666666667, f1:0.9628647214854111
1/1 [==============================] - 0s 50ms/step
no_hn:5,  af:relu, lf:categorical_crossentropy, lr:0.1, bs:10, acc:0.3, f1:0.15384615384615383
1/1 [==============================] - 0s 49ms/step
no_hn:5,  af:relu, lf:categorical_crossentropy, lr:0.1, bs:20, acc:0.9333333333333333, f1:0.9285714285714285
1/1 [==============================] - 0s 83ms/

In [27]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11490434/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [30]:
def create_mnist_model(no_hn, af, lf, lr, bs):

  model=Sequential()
  model.add(keras.Input(shape=input_shape))
  model.add(layers.Conv2D(no_hn, kernel_size=(3, 3), activation=af))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Conv2D(no_hn/2, kernel_size=(3, 3), activation=af))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Conv2D(no_hn/4, kernel_size=(3, 3), activation=af))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Flatten())
  model.add(layers.Dense(num_classes, activation="softmax"))
  opt = Adam(learning_rate=lr)
  epochs = 5
  model.compile(loss=lf, optimizer=opt, metrics=["accuracy"])
  model.fit(x_train, y_train, batch_size=bs, epochs=epochs, validation_split=0.1, verbose=0)

  prediction=model.predict(x_test)
  length=len(prediction)
  y_label=np.argmax(y_test,axis=1)
  predict_label=np.argmax(prediction,axis=1)

  acc = np.sum(y_label==predict_label)/length
  f1 = f1_score(y_label, predict_label, average='macro')
  conf_mat = confusion_matrix(y_label, predict_label)

  return acc, f1, conf_mat

In [31]:
acc, f1, conf_mat = create_mnist_model(32, 'relu', 'categorical_crossentropy', 0.01, 128)
print(acc)
print(f1)
print(conf_mat)

313/313 [==============================] - 2s 6ms/step
0.9571
0.9571213636367102
[[ 936    8    2    0    7    7    4    2    4   10]
 [   0 1128    4    0    0    0    0    2    1    0]
 [   1    8  967   16    4    1    2   19   13    1]
 [   0    0   19  972    0    5    0    9    4    1]
 [   7    5    3    0  946    0    3    2    1   15]
 [   2    3    0   14    0  862    1    2    6    2]
 [  10    1    4    0   13    2  920    0    8    0]
 [   0   18   39    9    0    1    0  958    0    3]
 [   3    1   21    2    2   11    2    0  926    6]
 [   4    0    2    2    9    9    0   12   15  956]]


In [32]:
# no_hn, af, lf, lr, bs
for no_f in [32, 16]:
  for af in ['relu', 'tanh']:
    for lf in ['categorical_crossentropy', 'categorical_hinge']:
      for lr in [0.05, 0.1]:
        for bs in [128, 256]:
          acc, f1, conf = create_mnist_model(no_f, af, lf, lr, bs)
          print('no_f:{},  af:{}, lf:{}, lr:{}, bs:{}, acc:{}, f1:{}'.format(no_f, af, lf, lr, bs, acc, f1))

313/313 [==============================] - 2s 5ms/step
no_f:32,  af:relu, lf:categorical_crossentropy, lr:0.05, bs:128, acc:0.8717, f1:0.8703450719928906
313/313 [==============================] - 2s 7ms/step
no_f:32,  af:relu, lf:categorical_crossentropy, lr:0.05, bs:256, acc:0.1028, f1:0.018643453028654337
313/313 [==============================] - 2s 5ms/step
no_f:32,  af:relu, lf:categorical_crossentropy, lr:0.1, bs:128, acc:0.1135, f1:0.0203861697350696
313/313 [==============================] - 2s 6ms/step
no_f:32,  af:relu, lf:categorical_crossentropy, lr:0.1, bs:256, acc:0.1135, f1:0.0203861697350696
313/313 [==============================] - 2s 6ms/step
no_f:32,  af:relu, lf:categorical_hinge, lr:0.05, bs:128, acc:0.0958, f1:0.01748494250775689
313/313 [==============================] - 2s 6ms/step
no_f:32,  af:relu, lf:categorical_hinge, lr:0.05, bs:256, acc:0.0892, f1:0.01637899375688579
313/313 [==============================] - 3s 8ms/step
no_f:32,  af:relu, lf:categorical

In [ ]:
## Public Repository : https://github.com/narendra974/AIMLOPS_IISC/tree/main
## ReadMe File : https://github.com/narendra974/AIMLOPS_IISC/blob/main/README.md
## New Branch Created : https://github.com/narendra974/AIMLOPS_IISC/tree/update_readme
## Pull Request : https://github.com/narendra974/AIMLOPS_IISC/pull/1 (This pull request was merged to the main branch)

## Problem 2 (5 marks)


1.  Write an essay to explain the MLOps Lifecycle, including the integration of DevOps, DataOps, and ModelOps. Save it in a file named "README.md" ("README.txt")
2.   Create a public github repository.
3.   Commit the file containing the essay (in "step 1") to the main branch.
4.   Create and checkout to a new branch.
5.   Edit the "README.md" file and make some changes.
6.   Commit the changes to the new branch and send a pull request to the main branch.

Share the github repository url in the assignment.

Make sure to not make any changes to the repository after the due date. Penalty will be same as the earlier and the last edited time will be considered for it.



```
# Write your answers here
```

